
# Example of the `aitlas` toolbox in the context of benchmarking a dataset

This notebook shows a sample implementation of a multi class image classification using the `aitlas` toolbox using the Eurosat dataset.

In [4]:
import matplotlib.pyplot as plt
import os
import numpy as npi
import pandas as pd

from itertools import compress
from aitlas.datasets import EurosatDataset
from aitlas.models import ResNet50
from aitlas.visualizations import display_image_labels
from aitlas.utils import image_loader
from aitlas.tasks import StratifiedSplitTask

## Define the splits and dataset paths

In [31]:
splits = [(10, 90), (20, 80), (30, 70), (40, 60), (50, 50), (60, 40), (70, 30), (80, 20), (90, 10)] # one tuple is (train, test)
dataset_path = "/media/hdd/EuroSAT/2750/" # where is the dataset on disk
trainset_path = "/media/hdd/EuroSAT/train.csv" # where to store the train IDs 
testset_path = "/media/hdd/EuroSAT/test.csv" # where to store the test IDs

## Loop through the splits, train and evaluate

In [35]:
results = [] # results accumulator

# iterate through the splits
for train, test in splits:
    # configure split task
    split_config = {
            "split": {
                "train": {
                    "ratio": train,
                    "file": trainset_path
                },
                "test": {
                    "ratio": test,
                    "file": testset_path
                }
            },
            "path": dataset_path
    }
    split_task = StratifiedSplitTask(None, split_config)
    split_task.run()
    
    # setup train set
    train_dataset_config = {
        "batch_size": 4,
        "shuffle": True,
        "num_workers": 4,
        "csv_file_path": trainset_path,
        "transforms": ["aitlas.transforms.ResizeCenterCropFlipHVToTensor"]
    }

    train_dataset = EurosatDataset(train_dataset_config)
    
    # setup test set
    test_dataset_config = {
        "batch_size": 4,
        "shuffle": False,
        "num_workers": 4,
        "csv_file_path": testset_path,
        "transforms": ["aitlas.transforms.ResizeCenterCropToTensor"]
    }

    test_dataset = EurosatDataset(test_dataset_config)
    print(f"Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")
    
    
    # setup model
    epochs = 1
    model_directory = "./experiments/eurosat/"
    model_config = {"num_classes": 10, "learning_rate": 0.001,"pretrained": True}
    model = ShallowCNNNet(model_config)
    model.prepare()

    ## training and evaluation
    model.train_and_evaluate_model(
        train_dataset=train_dataset,
        epochs=epochs,
        model_directory=model_directory,
        val_dataset=test_dataset,
        run_id='1',
    )
    
    # collect results
    results.append(model.running_metrics.accuracy())

2021-09-06 16:27:30,749 INFO Loading data...
2021-09-06 16:27:30,803 INFO Making splits...
2021-09-06 16:27:30,849 INFO And that's it!
2021-09-06 16:27:30,912 INFO Starting training.


Train size: 2700, Test size: 24300


training: 100%|██████████████████████████████████████████████████████████████████████| 675/675 [00:02<00:00, 233.09it/s]
2021-09-06 16:27:33,815 INFO epoch: 1, time: 3, loss:  2.07171
testing on train set: 100%|██████████████████████████████████████████████████████████| 675/675 [00:02<00:00, 334.30it/s]
2021-09-06 16:27:36,037 INFO F1_score Micro:0.2611111111111111, F1_score Macro:nan, F1_score Weighted:nan, F1_score per Class:0.3882896764252696, 0.6296743063932448, 0.2837301587301588, 0.26763990267639903, 0.007874015748031496, nan, nan, nan, nan, 0.04215456674473068
testing on validation set: 100%|███████████████████████████████████████████████████| 6075/6075 [00:16<00:00, 362.71it/s]
2021-09-06 16:27:52,790 INFO F1_score Micro:0.2480658436213992, F1_score Macro:nan, F1_score Weighted:nan, F1_score per Class:0.34512829112862653, 0.6531049250535332, 0.24295346523709516, 0.24146695715323166, 0.022579244463742945, nan, nan, nan, nan, 0.054751250329033954
2021-09-06 16:27:52,860 INFO fini

Train size: 5400, Test size: 21600


training: 100%|████████████████████████████████████████████████████████████████████| 1350/1350 [00:05<00:00, 242.55it/s]
2021-09-06 16:27:58,585 INFO epoch: 1, time: 5, loss:  1.97002
testing on train set: 100%|████████████████████████████████████████████████████████| 1350/1350 [00:03<00:00, 348.12it/s]
2021-09-06 16:28:02,538 INFO F1_score Micro:0.2574074074074074, F1_score Macro:nan, F1_score Weighted:nan, F1_score per Class:0.2540073982737361, nan, 0.28617571059431524, 0.08333333333333334, 0.011811023622047244, nan, 0.015625, nan, 0.38853188334157196, 0.5017921146953406
testing on validation set: 100%|███████████████████████████████████████████████████| 5400/5400 [00:15<00:00, 359.89it/s]
2021-09-06 16:28:17,546 INFO F1_score Micro:0.25722222222222224, F1_score Macro:nan, F1_score Weighted:nan, F1_score per Class:0.28693009118541035, nan, 0.28627611634575995, 0.05286738351254481, 0.007916872835230085, nan, 0.00684596577017115, nan, 0.3834221432800473, 0.5010667479427005
2021-09-06 1

...

training:  87%|██████████████████████████████████████████████████████████▉         | 4680/5400 [00:18<00:02, 249.94it/s]2021-09-06 16:31:21,792 INFO [1, 4700], loss:  6.26681


training: 100%|████████████████████████████████████████████████████████████████████| 5400/5400 [00:21<00:00, 246.98it/s]
2021-09-06 16:31:24,647 INFO epoch: 1, time: 22, loss:  1.75343
testing on train set: 100%|████████████████████████████████████████████████████████| 5400/5400 [00:14<00:00, 362.76it/s]
2021-09-06 16:31:39,607 INFO F1_score Micro:0.42740740740740746, F1_score Macro:0.3648052056445669, F1_score Weighted:0.37163089637796226, F1_score per Class:0.5447219983883964, 0.6283414774360448, 0.36834245893479345, 0.16397058823529415, 0.585240421702237, 0.3243602525756065, 0.000998502246630055, 0.5547163747388719, 0.4213139169286778, 0.05604606525911709
testing on validation set: 100%|███████████████████████████████████████████████████| 1350/1350 [00:03<00:00, 349.86it/s]
2021-09-06 16:31:43,469 INFO F1_score Micro:0.42814814814814817, F1_score Macro:nan, F1_score Weighted:nan, F1_score per Class:0.5303514376996805, 0.6329849012775842, 0.3621730382293763, 0.1508295625942685, 0.586

Train size: 24300, Test size: 2700


training:  77%|████████████████████████████████████████████████████▍               | 4689/6075 [00:18<00:05, 250.17it/s]2021-09-06 16:32:02,646 INFO [1, 4700], loss:  6.59313


training: 100%|████████████████████████████████████████████████████████████████████| 6075/6075 [00:24<00:00, 247.75it/s]
2021-09-06 16:32:08,220 INFO epoch: 1, time: 25, loss:  1.73072
testing on train set: 100%|████████████████████████████████████████████████████████| 6075/6075 [00:16<00:00, 360.42it/s]
2021-09-06 16:32:25,342 INFO F1_score Micro:0.4554320987654321, F1_score Macro:0.4128453569645953, F1_score Weighted:0.4278654505792667, F1_score per Class:0.5076792840730439, 0.6028241335044929, 0.4451069800070151, 0.13510600071864892, 0.3679639755548408, 0.32192648922686945, 0.16195372750642675, 0.6243844790230451, 0.3571104043616538, 0.6043980956699161
testing on validation set: 100%|█████████████████████████████████████████████████████| 675/675 [00:02<00:00, 330.79it/s]
2021-09-06 16:32:27,386 INFO F1_score Micro:0.43592592592592594, F1_score Macro:0.394042495700639, F1_score Weighted:0.40709269097214706, F1_score per Class:0.489795918367347, 0.6169724770642201, 0.3551401869158878,

## See the results

In [36]:
df = pd.DataFrame(zip(splits, [round(float(r["Accuracy"])*100, 2) for r in results]), columns=["Train/Test", "Accuracy"])

In [37]:
df

,Train/Test,Accuracy
0,"(10, 90)",24.81
1,"(20, 80)",25.72
2,"(30, 70)",26.70
3,"(40, 60)",29.46
4,"(50, 50)",28.50
5,"(60, 40)",35.00
6,"(70, 30)",33.98
7,"(80, 20)",42.81
8,"(90, 10)",43.59
